In [1]:
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
from drivers.tree_builder import TreeNeo
from drivers.graph_models import TreeNode, Order, Family, graph,Kingdom,Occurrence
from drivers.graph_models import Cell,Mex4km, countObjectsOf
from drivers.graph_models import pickNode
import matplotlib.pyplot as plt
import pandas as pd
import itertools as it
import numpy as np

## Use the ggplot style
plt.style.use('ggplot')

## Obtaining the predictors
In this case we will bring all the variables to start working with everything

In [2]:
from raster_api.tools import RasterData
from raster_api.models import raster_models_dic as models
from sketches.models import Country
from mesh.models import MexMesh

Mexico = Country.objects.filter(name__contains="exico").get()
mexican_cells = MexMesh.objects.filter(cell__intersects=Mexico.geom)

### Obtaining everything

In [4]:
datadict = { key : RasterData(models[key],border=Mexico.geom) for key,value in models.iteritems()}
## Without resamling (whole data)
pixel_size = 0.25
%time datacube_field = map(lambda raster : raster.rescale(pixel_size),datadict.itervalues())
datacube = datacube_field

CPU times: user 296 ms, sys: 24 ms, total: 320 ms
Wall time: 4min 33s


In [5]:
## Agregate by month
# Take mean of everything
cubes = map(lambda (k,v): np.mean(v.toNumpyArray(),axis=0), datadict.iteritems())

In [10]:
extractCoords = lambda (k,v) : v.getCoordinates() 
coords = extractCoords(datadict.iteritems().next())

In [12]:
## Oke I need a way to extract the dataframe, maybe aggregate it by mean 
dataframe_cube = map(lambda cube: pd.DataFrame(cube.flatten()),cubes)

In [13]:
datacube = pd.concat(dataframe_cube,axis=1)

In [14]:
datacube = pd.concat([datacube,coords],axis=1)

In [16]:
datacube.columns = datadict.keys() + list(coords.columns)

## Let´s save it for the posterity

In [19]:
datacube.to_csv('/outputs/sample_predictors.csv')